In [1]:
import os
os.chdir('../')
%pwd

'c:\\PEC-26\\MLOPS - Journey\\Projects\\End-to-End-Wine-Quality-Prediction'

In [2]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str

In [3]:
from wine_quality.constants import *
from wine_quality.utils.common import read_yaml, create_directories

class ConfiguartionManager:
    def __init__(self, config_filepath: Path = CONFIG_FILE_PATH, params_filepath: Path = PARAMS_FILE_PATH, schema_filepath: Path = SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def getModelEvaluationConfig(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        return ModelEvaluationConfig(
            root_dir = config.root_dir,
            test_data_path = config.test_data_path,
            model_path = config.model_path,
            all_params = params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name
        )

In [4]:
import os
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from urllib.parse import urlparse
import numpy as np
import joblib
from wine_quality.utils.common import save_json

class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def evaluateMetrics(self, actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2
    
    def saveResults(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        X_test = test_data.drop([self.config.target_column], axis = 1)
        y_test = test_data[self.config.target_column]

        predictions = model.predict(X_test)

        rmse, mae, r2 = self.evaluateMetrics(y_test, predictions)

        scores = {"rmse": rmse, "mae": mae, "r2_score": r2}
        save_json(path = Path(self.config.metric_file_name), data= scores)


In [5]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.getModelEvaluationConfig()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.saveResults()
except Exception as e:
    raise e

[2025-01-09 12:06:23,400: INFO: common yaml file: config\config.yaml loaded successfully]
[2025-01-09 12:06:23,403: INFO: common yaml file: params.yaml loaded successfully]
[2025-01-09 12:06:23,404: INFO: common yaml file: schema.yaml loaded successfully]
[2025-01-09 12:06:23,406: INFO: common created directory at: artifacts]
[2025-01-09 12:06:23,407: INFO: common created directory at: artifacts/model_evaluation]
[2025-01-09 12:06:23,508: INFO: common json file saved at: artifacts\model_evaluation\metrics.json]
